In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import random as rnd
rnd.seed(42)

In [73]:
# # ОБРАЗЕЦ 

# class DataPipeline:
    
#     def __init__(self):
#         """Параметры класса"""
#         self.medians = None
#         self.longitude_median = None
#         self.latitude_median = None
        
#     def fit(self, df):
#         """Сохранение статистик"""
        
#         # Расчет медиан
#         self.house_year_mode = df['HouseYear'].mode()[0]
#         self.rooms_median = df['Rooms'].median()
#         self.square_median = df['Square'].median()
#         self.kitchen_square_median = df['LifeSquare'].median()
#         self.lifq_square_median = df['KitchenSquare'].median()
#         self.house_floor_median = df['HouseFloor'].median()
        
#     def transform(self, df):
#         """Трансформация данных"""
        
#         # 1. Пропуски
#         empty_values = df.isna().sum()
#         for i, j in empty_values.items():
#             # отбираем столбы с пропусками
#             if j > 0:
#                 # если количество пропусков больше 40%, то такой столбец удаляем
#                 if j / df.shape[0] > 0.3:
#                     # значения убираем
#                     df.drop(i, axis=1, inplace=True)
#                 else:
#                     # остальное заменяем на медиану
#                     df[i] = df[i].fillna(df[i].median())


                    
#         # 2. Выбросы (outliers)

#         # Заменяем квартиры с комнатами меньше 1 и больше 5 и площадью меньше 100 квадратов на медианные значения по комнатам
#         index_rooms = df.query('(Rooms < 1) | ((Rooms >= 5) & (Square < 100))').index
#         df.loc[index_rooms, ['Rooms']] = self.rooms_median

#         # Заменим значения года постройки больше текущего года на моду
#         df.loc[df['HouseYear'] > datetime.now().year, 'HouseYear'] = self.house_year_mode

#         # округлим значения до 1 знака после запятой
#         df['Square'] = df['Square'].apply(lambda x: np.round(x, 1))
#         df['LifeSquare'] = df['LifeSquare'].apply(lambda x: np.round(x, 1))

#         # заменим аномально большие значение и значения меньше 5 на медиану
#         df.loc[((df['Square'] > 400) | (df['Square'] < 5)), ['Square']] = self.square_median
#         df.loc[(df['LifeSquare'] > df['Square'].max()), ['LifeSquare']] = self.kitchen_square_median
#         df.loc[(df['KitchenSquare'] > df['Square'].max()), ['KitchenSquare']] = self.lifq_square_median

#         # заменим все значения площади кухни больше 30 на медиану
#         df.loc[(df['KitchenSquare'] >= 30), ['KitchenSquare']] = self.lifq_square_median

#         # Если общая площадь меньше жилой, то ее значение меняем на LifeSquare = KitchenSquare
#         df.loc[(df['Square'] <= df['LifeSquare']), ['Square']] = df['LifeSquare'] + df['KitchenSquare']

#         # Если жилая площадь меньше кухни, то заменим ее на Square - KitchenSquare
#         df.loc[(df['LifeSquare'] <= df['KitchenSquare']), ['LifeSquare']] = df['Square'] - df['KitchenSquare']
#         df.loc[(df['KitchenSquare'] > (df['Square'] - df['LifeSquare'])), ['KitchenSquare']] = df['Square'] - df['LifeSquare']



#         df.loc[((df['HouseFloor'] > 50) | (df['HouseFloor'] < 1)), 'HouseFloor'] = self.house_floor_median

#         # заменим это значение на случайное число
#         index_floor = df.loc[(df['Floor'] > df['HouseFloor']), 'HouseFloor'].index
#         df.loc[index_floor, 'Floor'] = df.loc[index_floor, 'HouseFloor'].apply(lambda x: np.random.randint(1, x+1))

#         # # 3. Новые фичи (features)
        
#         # # Доля спален в общем кол-ве комнат
#         # df['bedroom_share'] = df['total_bedrooms'] / df['total_rooms'] * 100

#         # # Сколько человек в среднем живут в одной комнате
#         # df['population_per_room'] = df['population'] / df['total_rooms']
        
#         # # 4. Обработка категорий
#         # df = pd.concat([df, pd.get_dummies(df['ocean_proximity'])], axis=1)
        
#         return df


In [74]:
# # убираем пропуски
# def clear_empty_values(data):
#     # смотрим пустые значения
#     empty_values = data.isna().sum()
#     for i, j in empty_values.items():
#         # отбираем столбы с пропусками
#         if j > 0:
#             # если количество пропусков больше 40%, то такой столбец удаляем
#             if j / data.shape[0] > 0.3:
#                 # значения убираем
#                 data.drop(i, axis=1, inplace=True)
#             else:
#                 # остальное заменяем на медиану
#                 data[i] = data[i].fillna(data[i].median())
#     return data

In [75]:
# ОБРАЗЕЦ 

class DataPipeline:
    
    def __init__(self):
        """Параметры класса"""
        self.medians = None

    def fit(self, df):
        """Сохранение статистик"""
        
        # Расчет медиан
        self.house_year_mode = df['HouseYear'].mode()[0]
        self.rooms_median = df['Rooms'].median()
        self.square_median = df['Square'].median()
        self.kitchen_square_median = df['KitchenSquare'].median()
        self.life_square_median = df['LifeSquare'].median()
        self.house_floor_median = df['HouseFloor'].median()
        
    # убираем пропуски
    def clear_empty_values(self, df):
        # смотрим пустые значения
        empty_values = df.isna().sum()
        for i, j in empty_values.items():
            # отбираем столбы с пропусками
            if j > 0:
                # если количество пропусков больше 40%, то такой столбец удаляем
                if j / df.shape[0] > 0.3:
                    # значения убираем
                    df.drop(i, axis=1, inplace=True)
                else:
                    # остальное заменяем на медиану
                    df[i] = df[i].fillna(df[i].median())
        return df

    def transform(self, df):
        """Трансформация данных"""
       
        # 2. Выбросы (outliers)

        # Заменяем квартиры с комнатами меньше 1 и больше 5 и площадью меньше 100 квадратов на медианные значения по комнатам
        index_rooms = df.query('(Rooms < 1) | ((Rooms >= 5) & (Square < 100))').index
        df.loc[index_rooms, ['Rooms']] = self.rooms_median

        # Заменим значения года постройки больше текущего года на моду
        df.loc[df['HouseYear'] > datetime.now().year, 'HouseYear'] = self.house_year_mode

        # округлим значения до 1 знака после запятой
        df['Square'] = df['Square'].apply(lambda x: np.round(x, 1))
        df['LifeSquare'] = df['LifeSquare'].apply(lambda x: np.round(x, 1))

        # заменим аномально большие значение и значения меньше 5 на медиану
        df.loc[((df['Square'] > 400) | (df['Square'] < 5)), ['Square']] = self.square_median
        df.loc[(df['LifeSquare'] > df['Square'].max()), ['LifeSquare']] = self.life_square_median 
        df.loc[(df['KitchenSquare'] > df['Square'].max()), ['KitchenSquare']] = self.kitchen_square_median

        # заменим все значения площади кухни больше 30 на медиану
        df.loc[(df['KitchenSquare'] >= 30), ['KitchenSquare']] = self.kitchen_square_median

        # Если общая площадь меньше жилой, то ее значение меняем на LifeSquare = KitchenSquare
        df.loc[(df['Square'] <= df['LifeSquare']), ['Square']] = df['LifeSquare'] + df['KitchenSquare']

        # Если жилая площадь меньше кухни, то заменим ее на Square - KitchenSquare
        df.loc[(df['LifeSquare'] <= df['KitchenSquare']), ['LifeSquare']] = df['Square'] - df['KitchenSquare']

        df.loc[(df['KitchenSquare'] > (df['Square'] - df['LifeSquare'])), ['KitchenSquare']] = df['Square'] - df['LifeSquare']


        # заменим дома больше 50 этажей и меньше 1 на медианное значение по этажам
        df.loc[((df['HouseFloor'] > 50) | (df['HouseFloor'] < 1)), 'HouseFloor'] = self.house_floor_median
        rnd.seed(42)


        index_floor = df.loc[(df['Floor'] > df['HouseFloor'])].index
        df.loc[index_floor, 'Floor'] = df.loc[index_floor, 'HouseFloor'].apply(lambda x: rnd.randint(1, x))

        # # 3. Новые фичи (features)
        
        # # Доля спален в общем кол-ве комнат
        # df['bedroom_share'] = df['total_bedrooms'] / df['total_rooms'] * 100

        # # Сколько человек в среднем живут в одной комнате
        # df['population_per_room'] = df['population'] / df['total_rooms']
        
        # # 4. Обработка категорий
        # df = pd.concat([df, pd.get_dummies(df['ocean_proximity'])], axis=1)
        
        return df


In [76]:
TRAIN_DATA = './train.csv'

In [77]:
train_data_new = pd.read_csv(TRAIN_DATA, engine='python', on_bad_lines='skip')
train_data_new.head()


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [78]:
SquareList = ['Square', 'LifeSquare', 'KitchenSquare']

In [79]:
train_data_new[SquareList].iloc[1316]

4.633498405895908

In [80]:
# Создадим объект класса
dp = DataPipeline()

In [81]:
dp.fit(train_data_new)

In [82]:
proverka = dp.transform(train_data_new)

In [83]:
proverka[SquareList].iloc[1316]

52.51330992401352

In [84]:
proverka[SquareList].iloc[1316].equals(train_data_new[SquareList].iloc[1316])

True

In [85]:
# for i in range(0, 10000):
#     if not proverka[SquareList].iloc[i].equals(train_data[SquareList].iloc[i]):
#         print(i)